In [356]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

pd.set_option('display.max_columns', None)

In [357]:
all_games_df = pd.read_csv('all_games_df.csv')
all_games_df = all_games_df[['is_tourney','Season','HTeamID','RTeamID','Hwin']]
all_games_df.head()

,is_tourney,Season,HTeamID,RTeamID,Hwin
0,0.0,2004,1113.0,1168.0,1
1,0.0,2004,1337.0,1168.0,0
2,0.0,2004,1414.0,1168.0,1
3,0.0,2004,1142.0,1168.0,1
4,0.0,2004,1429.0,1168.0,1


In [358]:
all_games_df.tail()

,is_tourney,Season,HTeamID,RTeamID,Hwin
57373,0.0,2020,1285.0,1294.0,0
57374,0.0,2020,1286.0,1294.0,0
57375,0.0,2020,1319.0,1294.0,0
57376,0.0,2020,1381.0,1294.0,0
57377,0.0,2020,1226.0,1294.0,0


In [359]:
kp_df = pd.read_csv('kp_all.csv')
kp_df.sort_index()

,team,conf,adjem,adjo,adjd,luck,Season,TeamID
0,michigan st.,B10,24.03,115.2,91.2,-0.012,2020,1277
1,michigan st.,B10,31.36,121.7,90.4,-0.007,2019,1277
2,michigan st.,B10,25.41,119.2,93.8,0.038,2018,1277
3,michigan st.,B10,15.51,111.4,95.9,-0.011,2017,1277
4,michigan st.,B10,27.97,123.4,95.5,-0.012,2016,1277
...,...,...,...,...,...,...,...,...
6155,troy st.,ASun,6.15,106.9,100.8,0.105,2003,1407
6156,troy st.,ASun,0.15,104.2,104.1,0.044,2002,1407
6157,middle tennessee st.,SB,3.36,103.8,100.5,-0.006,2003,1292
6158,morris brown,ind,-27.55,86.9,114.4,0.003,2003,1289


In [360]:
kp_df.tail()

,team,conf,adjem,adjo,adjd,luck,Season,TeamID
6155,troy st.,ASun,6.15,106.9,100.8,0.105,2003,1407
6156,troy st.,ASun,0.15,104.2,104.1,0.044,2002,1407
6157,middle tennessee st.,SB,3.36,103.8,100.5,-0.006,2003,1292
6158,morris brown,ind,-27.55,86.9,114.4,0.003,2003,1289
6159,morris brown,ind,-29.95,85.1,115.1,0.044,2002,1289


In [361]:
regseason_df = pd.read_csv('regseason_df.csv')
regseason_df  = regseason_df[['TeamID','Season','wins_top25','PointMargin','FG','FG3']]
regseason_df.head()

,TeamID,Season,wins_top25,PointMargin,FG,FG3
0,1101,2014,0,-15.476190,0.410184,0.380275
1,1101,2015,0,-10.857143,0.407501,0.381507
2,1101,2016,0,-6.407407,0.442606,0.364517
3,1101,2017,0,-4.760000,0.457867,0.371026
4,1101,2018,0,-2.148148,0.452366,0.327240


In [362]:
regseason_df.tail()

,TeamID,Season,wins_top25,PointMargin,FG,FG3
6182,1465,2019,0,0.269231,0.444370,0.351052
6183,1465,2020,0,4.666667,0.430507,0.374884
6184,1466,2019,0,-9.310345,0.384144,0.295101
6185,1466,2020,0,-3.178571,0.421914,0.344007
6186,1467,2020,0,-0.400000,0.416995,0.334847


In [363]:
massey_df = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM/MMasseyOrdinals.csv')
POM_df = massey_df[massey_df['SystemName'].str.contains("POM")]
POM_end_df = POM_df.loc[POM_df['RankingDayNum'] == 133]
POM_end_df.rename(columns={'OrdinalRank': 'RankPOM'}, inplace=True)
POM_end_df = POM_end_df[['Season','TeamID','RankPOM']]
POM_end_df.head()

,Season,TeamID,RankPOM
127259,2003,1102,160
127260,2003,1103,163
127261,2003,1104,33
127262,2003,1105,307
127263,2003,1106,263


In [364]:
POM_end_df.tail()

,Season,TeamID,RankPOM
3812063,2019,1462,64
3812064,2019,1463,82
3812065,2019,1464,263
3812066,2019,1465,192
3812067,2019,1466,285


Create test data set 

In [365]:
#Test set (this sets the data up in the format Kaggle needs for scoring)
df_seeds = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM/MNCAATourneySeeds.csv')
df_seeds = df_seeds[df_seeds['Season']>=2015]
df_19_tourney = df_seeds.merge(df_seeds, how='inner', on='Season')
df_19_tourney = df_19_tourney[df_19_tourney['TeamID_x'] < df_19_tourney['TeamID_y']]
df_19_tourney['ID'] = df_19_tourney['Season'].astype(str) + '_' \
              + df_19_tourney['TeamID_x'].astype(str) + '_' \
              + df_19_tourney['TeamID_y'].astype(str)
df_19_tourney['SeedInt_x'] = [int(x[1:3]) for x in df_19_tourney['Seed_x']]
df_19_tourney['SeedInt_y'] = [int(x[1:3]) for x in df_19_tourney['Seed_y']]

#Make home team lower seed (consistent with training data)
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_y']

df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_y']

df_19_tourney['is_tourney'] = 1

df_19_tourney = df_19_tourney.drop(['Seed_x','Seed_y','TeamID_x','TeamID_y','SeedInt_x','SeedInt_y'], axis=1)
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney
1,2015,2015_1437_1438,1437.0,1438.0,1.0,2.0,1
12,2015,2015_1437_1461,1437.0,1461.0,1.0,12.0,1
38,2015,2015_1437_1452,1437.0,1452.0,1.0,5.0,1
40,2015,2015_1437_1455,1437.0,1455.0,1.0,7.0,1
51,2015,2015_1437_1458,1437.0,1458.0,1.0,1.0,1
...,...,...,...,...,...,...,...
23113,2019,2019_1205_1234,1234.0,1205.0,10.0,16.0,1
23114,2019,2019_1205_1388,1388.0,1205.0,11.0,16.0,1
23115,2019,2019_1205_1332,1332.0,1205.0,12.0,16.0,1
23116,2019,2019_1205_1414,1414.0,1205.0,13.0,16.0,1


In [366]:
home_road = ['H','R']
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, regseason_df, left_on=['Season',hr+'TeamID'], right_on = ['Season','TeamID'], how='left')
    df_19_tourney.rename(columns={'wins_top25': hr+'wins_top25'}, inplace=True)
    df_19_tourney.rename(columns={'PointMargin': hr+'PointMargin'}, inplace=True)
    df_19_tourney.rename(columns={'FG': hr+'FG'}, inplace=True)
    df_19_tourney.rename(columns={'FG3': hr+'FG3'}, inplace=True)
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)

In [367]:
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3
0,2015,2015_1437_1438,1437.0,1438.0,1.0,2.0,1,3,15.382353,0.470578,0.386226,3,14.593750,0.465041,0.365095
1,2015,2015_1437_1461,1437.0,1461.0,1.0,12.0,1,3,15.382353,0.470578,0.386226,0,4.500000,0.459152,0.323069
2,2015,2015_1437_1452,1437.0,1452.0,1.0,5.0,1,3,15.382353,0.470578,0.386226,3,7.093750,0.412337,0.324571
3,2015,2015_1437_1455,1437.0,1455.0,1.0,7.0,1,3,15.382353,0.470578,0.386226,1,12.806452,0.445620,0.349481
4,2015,2015_1437_1458,1437.0,1458.0,1.0,1.0,1,3,15.382353,0.470578,0.386226,7,15.764706,0.479012,0.357805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2019,2019_1205_1234,1234.0,1205.0,10.0,16.0,1,2,4.666667,0.455846,0.358046,0,2.354839,0.477831,0.381009
11386,2019,2019_1205_1388,1388.0,1205.0,11.0,16.0,1,1,8.545455,0.475913,0.377759,0,2.354839,0.477831,0.381009
11387,2019,2019_1205_1332,1332.0,1205.0,12.0,16.0,1,0,7.542857,0.452262,0.348826,0,2.354839,0.477831,0.381009
11388,2019,2019_1205_1414,1414.0,1205.0,13.0,16.0,1,0,8.852941,0.456285,0.358201,0,2.354839,0.477831,0.381009


In [368]:
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, POM_end_df, left_on=['Season',hr+'TeamID'], right_on = ['Season','TeamID'], how='left')
    df_19_tourney.rename(columns={'RankPOM': hr+'RankPOM'}, inplace=True)
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)

In [369]:
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM
0,2015,2015_1437_1438,1437.0,1438.0,1.0,2.0,1,3,15.382353,0.470578,0.386226,3,14.593750,0.465041,0.365095,5,4
1,2015,2015_1437_1461,1437.0,1461.0,1.0,12.0,1,3,15.382353,0.470578,0.386226,0,4.500000,0.459152,0.323069,5,102
2,2015,2015_1437_1452,1437.0,1452.0,1.0,5.0,1,3,15.382353,0.470578,0.386226,3,7.093750,0.412337,0.324571,5,25
3,2015,2015_1437_1455,1437.0,1455.0,1.0,7.0,1,3,15.382353,0.470578,0.386226,1,12.806452,0.445620,0.349481,5,14
4,2015,2015_1437_1458,1437.0,1458.0,1.0,1.0,1,3,15.382353,0.470578,0.386226,7,15.764706,0.479012,0.357805,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2019,2019_1205_1234,1234.0,1205.0,10.0,16.0,1,2,4.666667,0.455846,0.358046,0,2.354839,0.477831,0.381009,36,167
11386,2019,2019_1205_1388,1388.0,1205.0,11.0,16.0,1,1,8.545455,0.475913,0.377759,0,2.354839,0.477831,0.381009,31,167
11387,2019,2019_1205_1332,1332.0,1205.0,12.0,16.0,1,0,7.542857,0.452262,0.348826,0,2.354839,0.477831,0.381009,43,167
11388,2019,2019_1205_1414,1414.0,1205.0,13.0,16.0,1,0,8.852941,0.456285,0.358201,0,2.354839,0.477831,0.381009,75,167


In [370]:
efficiency_list = ['conf','adjem','adjo','adjd','luck', 'TeamID']
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, kp_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)
    for metric in efficiency_list:
        df_19_tourney.rename(columns={metric: hr+metric}, inplace=True)
    if hr == 'H':
        df_19_tourney.rename(columns={'team': 'home'}, inplace=True)
    if hr == 'R':
        df_19_tourney.rename(columns={'team': 'road'}, inplace=True)

In [371]:
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck
0,2015,2015_1437_1438,1437.0,1438.0,1.0,2.0,1,3,15.382353,0.470578,0.386226,3,14.593750,0.465041,0.365095,5,4,villanova,BE,30.65,122.7,92.0,0.031,virginia,ACC,30.06,115.6,85.5,0.023
1,2015,2015_1437_1461,1437.0,1461.0,1.0,12.0,1,3,15.382353,0.470578,0.386226,0,4.500000,0.459152,0.323069,5,102,villanova,BE,30.65,122.7,92.0,0.031,wyoming,MWC,5.67,104.8,99.1,0.083
2,2015,2015_1437_1452,1437.0,1452.0,1.0,5.0,1,3,15.382353,0.470578,0.386226,3,7.093750,0.412337,0.324571,5,25,villanova,BE,30.65,122.7,92.0,0.031,west virginia,B12,17.64,112.6,94.9,0.087
3,2015,2015_1437_1455,1437.0,1455.0,1.0,7.0,1,3,15.382353,0.470578,0.386226,1,12.806452,0.445620,0.349481,5,14,villanova,BE,30.65,122.7,92.0,0.031,wichita st.,MVC,22.21,115.8,93.6,0.022
4,2015,2015_1437_1458,1437.0,1458.0,1.0,1.0,1,3,15.382353,0.470578,0.386226,7,15.764706,0.479012,0.357805,5,3,villanova,BE,30.65,122.7,92.0,0.031,wisconsin,B10,33.72,129.0,95.2,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2019,2019_1205_1234,1234.0,1205.0,10.0,16.0,1,2,4.666667,0.455846,0.358046,0,2.354839,0.477831,0.381009,36,167,iowa,B10,15.96,117.2,101.2,0.057,gardner webb,BSth,-0.74,107.5,108.2,0.070
11386,2019,2019_1205_1388,1388.0,1205.0,11.0,16.0,1,1,8.545455,0.475913,0.377759,0,2.354839,0.477831,0.381009,31,167,saint mary's,WCC,17.71,114.6,96.9,-0.033,gardner webb,BSth,-0.74,107.5,108.2,0.070
11387,2019,2019_1205_1332,1332.0,1205.0,12.0,16.0,1,0,7.542857,0.452262,0.348826,0,2.354839,0.477831,0.381009,43,167,oregon,P12,15.15,107.7,92.5,-0.039,gardner webb,BSth,-0.74,107.5,108.2,0.070
11388,2019,2019_1205_1414,1414.0,1205.0,13.0,16.0,1,0,8.852941,0.456285,0.358201,0,2.354839,0.477831,0.381009,75,167,uc irvine,BW,9.46,106.6,97.1,0.097,gardner webb,BSth,-0.74,107.5,108.2,0.070


In [372]:
df_19_tourney['Htourny20plus'] = 0
df_19_tourney['Rtourny20plus'] = 0

experienced_teams = ['kansas','north carolina','kentucky','duke','michigan st.','wisconsin','florida','villanova','gonzaga','louisville','arizona','xavier','connecticut','syracuse','butler','ohio st.','ucla','west virginia','texas','michigan','pittsburgh','memphis','oregon']
for team in experienced_teams:
    df_19_tourney.loc[df_19_tourney['home']==team, 'Htourny20plus'] = 1
    df_19_tourney.loc[df_19_tourney['road']==team, 'Rtourny20plus'] = 1

In [373]:
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck,Htourny20plus,Rtourny20plus
0,2015,2015_1437_1438,1437.0,1438.0,1.0,2.0,1,3,15.382353,0.470578,0.386226,3,14.593750,0.465041,0.365095,5,4,villanova,BE,30.65,122.7,92.0,0.031,virginia,ACC,30.06,115.6,85.5,0.023,1,0
1,2015,2015_1437_1461,1437.0,1461.0,1.0,12.0,1,3,15.382353,0.470578,0.386226,0,4.500000,0.459152,0.323069,5,102,villanova,BE,30.65,122.7,92.0,0.031,wyoming,MWC,5.67,104.8,99.1,0.083,1,0
2,2015,2015_1437_1452,1437.0,1452.0,1.0,5.0,1,3,15.382353,0.470578,0.386226,3,7.093750,0.412337,0.324571,5,25,villanova,BE,30.65,122.7,92.0,0.031,west virginia,B12,17.64,112.6,94.9,0.087,1,1
3,2015,2015_1437_1455,1437.0,1455.0,1.0,7.0,1,3,15.382353,0.470578,0.386226,1,12.806452,0.445620,0.349481,5,14,villanova,BE,30.65,122.7,92.0,0.031,wichita st.,MVC,22.21,115.8,93.6,0.022,1,0
4,2015,2015_1437_1458,1437.0,1458.0,1.0,1.0,1,3,15.382353,0.470578,0.386226,7,15.764706,0.479012,0.357805,5,3,villanova,BE,30.65,122.7,92.0,0.031,wisconsin,B10,33.72,129.0,95.2,0.015,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2019,2019_1205_1234,1234.0,1205.0,10.0,16.0,1,2,4.666667,0.455846,0.358046,0,2.354839,0.477831,0.381009,36,167,iowa,B10,15.96,117.2,101.2,0.057,gardner webb,BSth,-0.74,107.5,108.2,0.070,0,0
11386,2019,2019_1205_1388,1388.0,1205.0,11.0,16.0,1,1,8.545455,0.475913,0.377759,0,2.354839,0.477831,0.381009,31,167,saint mary's,WCC,17.71,114.6,96.9,-0.033,gardner webb,BSth,-0.74,107.5,108.2,0.070,0,0
11387,2019,2019_1205_1332,1332.0,1205.0,12.0,16.0,1,0,7.542857,0.452262,0.348826,0,2.354839,0.477831,0.381009,43,167,oregon,P12,15.15,107.7,92.5,-0.039,gardner webb,BSth,-0.74,107.5,108.2,0.070,1,0
11388,2019,2019_1205_1414,1414.0,1205.0,13.0,16.0,1,0,8.852941,0.456285,0.358201,0,2.354839,0.477831,0.381009,75,167,uc irvine,BW,9.46,106.6,97.1,0.097,gardner webb,BSth,-0.74,107.5,108.2,0.070,0,0


In [374]:
df_19_tourney['HBig4Conf'] = 0
df_19_tourney['RBig4Conf'] = 0
conferences = ['ACC','B10','B12','SEC']
for conf in conferences:
    df_19_tourney.loc[df_19_tourney['Hconf']==conf, 'HBig4Conf'] = 1
    df_19_tourney.loc[df_19_tourney['Rconf']==conf, 'RBig4Conf'] = 1

In [375]:
list(df_19_tourney)

['Season',
 'ID',
 'HTeamID',
 'RTeamID',
 'HSeed',
 'RSeed',
 'is_tourney',
 'Hwins_top25',
 'HPointMargin',
 'HFG',
 'HFG3',
 'Rwins_top25',
 'RPointMargin',
 'RFG',
 'RFG3',
 'HRankPOM',
 'RRankPOM',
 'home',
 'Hconf',
 'Hadjem',
 'Hadjo',
 'Hadjd',
 'Hluck',
 'road',
 'Rconf',
 'Radjem',
 'Radjo',
 'Radjd',
 'Rluck',
 'Htourny20plus',
 'Rtourny20plus',
 'HBig4Conf',
 'RBig4Conf']

In [376]:
df_19_tourney.sort_index()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck,Htourny20plus,Rtourny20plus,HBig4Conf,RBig4Conf
0,2015,2015_1437_1438,1437.0,1438.0,1.0,2.0,1,3,15.382353,0.470578,0.386226,3,14.593750,0.465041,0.365095,5,4,villanova,BE,30.65,122.7,92.0,0.031,virginia,ACC,30.06,115.6,85.5,0.023,1,0,0,1
1,2015,2015_1437_1461,1437.0,1461.0,1.0,12.0,1,3,15.382353,0.470578,0.386226,0,4.500000,0.459152,0.323069,5,102,villanova,BE,30.65,122.7,92.0,0.031,wyoming,MWC,5.67,104.8,99.1,0.083,1,0,0,0
2,2015,2015_1437_1452,1437.0,1452.0,1.0,5.0,1,3,15.382353,0.470578,0.386226,3,7.093750,0.412337,0.324571,5,25,villanova,BE,30.65,122.7,92.0,0.031,west virginia,B12,17.64,112.6,94.9,0.087,1,1,0,1
3,2015,2015_1437_1455,1437.0,1455.0,1.0,7.0,1,3,15.382353,0.470578,0.386226,1,12.806452,0.445620,0.349481,5,14,villanova,BE,30.65,122.7,92.0,0.031,wichita st.,MVC,22.21,115.8,93.6,0.022,1,0,0,0
4,2015,2015_1437_1458,1437.0,1458.0,1.0,1.0,1,3,15.382353,0.470578,0.386226,7,15.764706,0.479012,0.357805,5,3,villanova,BE,30.65,122.7,92.0,0.031,wisconsin,B10,33.72,129.0,95.2,0.015,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2019,2019_1205_1234,1234.0,1205.0,10.0,16.0,1,2,4.666667,0.455846,0.358046,0,2.354839,0.477831,0.381009,36,167,iowa,B10,15.96,117.2,101.2,0.057,gardner webb,BSth,-0.74,107.5,108.2,0.070,0,0,1,0
11386,2019,2019_1205_1388,1388.0,1205.0,11.0,16.0,1,1,8.545455,0.475913,0.377759,0,2.354839,0.477831,0.381009,31,167,saint mary's,WCC,17.71,114.6,96.9,-0.033,gardner webb,BSth,-0.74,107.5,108.2,0.070,0,0,0,0
11387,2019,2019_1205_1332,1332.0,1205.0,12.0,16.0,1,0,7.542857,0.452262,0.348826,0,2.354839,0.477831,0.381009,43,167,oregon,P12,15.15,107.7,92.5,-0.039,gardner webb,BSth,-0.74,107.5,108.2,0.070,1,0,0,0
11388,2019,2019_1205_1414,1414.0,1205.0,13.0,16.0,1,0,8.852941,0.456285,0.358201,0,2.354839,0.477831,0.381009,75,167,uc irvine,BW,9.46,106.6,97.1,0.097,gardner webb,BSth,-0.74,107.5,108.2,0.070,0,0,0,0


In [377]:
df_19_tourney = df_19_tourney.fillna(df_19_tourney.mean())

In [378]:
df_19_tourney.to_csv('test_combos_df_2015.csv', index=False)